Electricity showcase for basic PULPO

Written by Fabian Lechtenberg, 18.09.2023

Last Update: 23.09.2023

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">(1) Selection of LCI Data</h2>
</div>

### Import section

In this working version of the pulpo repository, pulpo musst be imported from the folder above, which can be done by appending ".." to the system path. Later, only the import after installation via pip will be necessary.

In [1]:
import os
import sys
sys.path.append('../')
from pulpo import pulpo

import pandas as pd
pd.set_option('display.max_colwidth', None)

### Setup

Specify the project, database and method to be used. Also indicate the folder where the working data should be stored. For this example to work, you need a project "**pulpo**" with a database "**cutoff38**", which is the ecoinvent 3.8 cutoff system model.

In [2]:
project = "pulpo"
database = "cutoff38"
methods = "('IPCC 2013', 'climate change', 'GWP 100a')"

# Substitute with your working directory of choice
notebook_dir = os.path.dirname(os.getcwd())
directory = os.path.join(notebook_dir, 'data')

# Substitute with your GAMS path
GAMS_PATH = "C:/GAMS/37/gams.exe"

Create a pulpo object called "pulpo_worker". This object is an element of the class "PulpoOptimizer", a class that links the different utilitiy modules containing code for retrieving, preparing and adjusting the data, preparing and running the optimization problem, as well as saving the results.

In [3]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, methods, directory)

Retrieve the data. If data is already loaded, this step is automatically skipped. 

In [4]:
pulpo_worker.get_lci_data()

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">(2) User Specifications</h2>
</div>

<div style="text-align: center;">
    <img src="pictures/electricity_showcase_1.png" alt="Image Alt Text" style="width: 50%;" />
</div>

### Specify the **functional unit**

Retrieve the market activity for liquid hydrogen in Europe (RER). Use the function "**<span style="color: red;">retrieve_activities</span>**" for this purpose. The function takes 4 optional arguments: "keys" (🔑) --> "activities" (⚙️) --> "reference_products" (📦) --> "locations" (🗺️). The activities are retrieved by this order. 

Since the key is unique, a single activity for each passed key will be returned. Activity names, reference_prduct and locations are not unique, so the best match for the passed data will be returned. 

#### Passing keys  🔑

Keys can be obtained e.g. directly from **activity browser** and several keys can be passed at the same time.

In [5]:
keys = ["('cutoff38', '962727b9a36bcaa186f222b29b57f6a3')", "('cutoff38', '473d4bb488e8f903b58203f3e5161636')"]

In [6]:
pulpo_worker.retrieve_activities(keys=keys)

['market for electricity, high voltage' (kilowatt hour, ES, None),
 'market for electricity, high voltage' (kilowatt hour, DE, None)]

#### Passing activity  name (⚙️), reference_product (📦) and/or location (🗺️)

Instead of passing the keys, a combination of activities, reference_products and locations can be passed. A best match (all existing combinations) will be returned. 

In [7]:
activities = ["market for electricity, high voltage"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]

It is also possible to pass only partial information such as only reference product or only activity name:

In [8]:
pulpo_worker.retrieve_activities(activities=activities)

['market for electricity, high voltage' (kilowatt hour, FI, None),
 'market for electricity, high voltage' (kilowatt hour, RO, None),
 'market for electricity, high voltage' (kilowatt hour, RS, None),
 'market for electricity, high voltage' (kilowatt hour, LB, None),
 'market for electricity, high voltage' (kilowatt hour, SS, None),
 'market for electricity, high voltage' (kilowatt hour, YE, None),
 'market for electricity, high voltage' (kilowatt hour, BR-North-eastern grid, None),
 'market for electricity, high voltage' (kilowatt hour, HT, None),
 'market for electricity, high voltage' (kilowatt hour, AU, None),
 'market for electricity, high voltage' (kilowatt hour, DK, None),
 'market for electricity, high voltage' (kilowatt hour, TT, None),
 'market for electricity, high voltage' (kilowatt hour, TZ, None),
 'market for electricity, high voltage' (kilowatt hour, US-NPCC, None),
 'market for electricity, high voltage' (kilowatt hour, ES, None),
 'market for electricity, high voltage

In [9]:
pulpo_worker.retrieve_activities(reference_products=reference_products)

['electricity production, hard coal' (kilowatt hour, CN-HE, None),
 'heat and power co-generation, biogas, gas engine' (kilowatt hour, CA-AB, None),
 'electricity production, hydro, pumped storage' (kilowatt hour, CA-NS, None),
 'heat and power co-generation, oil' (kilowatt hour, ES, None),
 'treatment of blast furnace gas, in power plant' (kilowatt hour, CN-HU, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CO, None),
 'market for electricity, high voltage' (kilowatt hour, PE, None),
 'market for electricity, high voltage' (kilowatt hour, BH, None),
 'electricity, high voltage, import from MK' (kilowatt hour, BG, None),
 'electricity production, hydro, run-of-river' (kilowatt hour, IE, None),
 'electricity production, hydro, reservoir, non-alpine region' (kilowatt hour, TZ, None),
 'heat and power co-generation, biogas, gas engine' (kilowatt hour, LU, None),
 'electricity production, oil' (kilowatt hour, ZA, None),
 'ele

Let's retrieve the activity of our functional unit and specify the demand as a dictionary:

In [10]:
electricity_market = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

In [11]:
electricity_market

['market for electricity, high voltage' (kilowatt hour, DE, None)]

Setting a demand of 128,819.0 GWh of electricity according to [Germany electricity demand 2018](https://www.destatis.de/EN/Themes/Society-Environment/Environment/Material-Energy-Flows/Tables/electricity-consumption-households.html)


In [12]:
demand = {electricity_market[0]: 1.28819e+11}

### Specify the **choices**

The choices are specified similar to the demand / functional unit. First, search for the processes with equivalent products:

In [13]:
activities = ["electricity production, lignite", 
             "electricity production, hard coal",
             "electricity production, nuclear, pressure water reactor",
             "electricity production, wind, 1-3MW turbine, onshore"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]

electricity_activities = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

In [14]:
electricity_activities

['electricity production, hard coal' (kilowatt hour, DE, None),
 'electricity production, nuclear, pressure water reactor' (kilowatt hour, DE, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, DE, None),
 'electricity production, lignite' (kilowatt hour, DE, None)]

These are the currently four most employed technologies for electricity production in Germany (lignite: 24.2%, wind: 15.4%, coal: 11.9%, nuclear: 10.6%) according to the "_market for electricity, high voltage (DE)_"

Specify also the choices as a dictionary. Be aware, that this time we are dealing with a dictionary of dictionaries. Each inner dictionary corresponds to one type of choice in the background! Here, we only consider choices between electricity production activities, so we assign the key "electricity" to the equivalent product they produce. 

The assigned value in the inner dictionary is the capacity limit of this activity, which for now is set to a very high value, to consider an unconstrained situation. 

In [15]:
choices  = {'electricity': {electricity_activities[0]: 1e16,
                         electricity_activities[1]: 1e16,
                         electricity_activities[2]: 1e16,
                         electricity_activities[3]: 1e16}}

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">(3) Solution</h2>
</div>

### Instantiate the worker

In [16]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand)

Creating Instance
Instance created


### Solve the instance

When specifying a valid GAMS_PATH with a licence for CPLEX, as shown below, CPLEX with fine-tuned parameters is automatically selected to solve the Linear Problem (**LP**).

If no GAMS_PATH is specified, the "[HiGHS](https://highs.dev/)" solver is automatically used. It has almost double the run time of "CPLEX".

In [ ]:
#results = pulpo_worker.solve()
# Alternatively using GAMS (cplex) solvers:
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

GAMS solvers library availability: True
Solver path: C:\GAMS\37\gams.exe
        2.26 seconds required for presolve
--- Job model.gms Start 12/11/23 15:28:57 37.1.0 r07954d5 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\37\gmsprmNT.txt
    C:\Users\Usuario\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIP CPLEX
    Input C:\Users\Usuario\AppData\Local\Temp\tmpj48fpzz7\model.gms
    Output C:\Users\Usuario\AppData\Local\Temp\tmpj48fpzz7\output.lst
    ScrDir C:\Users\Usuario\AppData\Local\Temp\tmpj48fpzz7\225a\
    SysDir C:\GAMS\37\
    CurDir C:\Users\Usuario\AppData\Local\Temp\tmpj48fpzz7\
    LogOption 3
    OptCR 1E-9
Licensee: Antonio Espuna, Single User License            S210319|0002AN-WIN
          Universitat Politecnica de Catalunya, Chemical Engineering DC6757
          C:\Users\Usuario\Documents\GAMS\gamslice.txt
          antonio.espuna@upc.edu                                           
Processor information: 1 socket(s), 12 core(s), and 20 thread

Iteration:  3719   Scaled dual infeas =       8602702.736013
Iteration:  3914   Scaled dual infeas =          2672.586834
Elapsed time = 19.58 sec. (50814.96 ticks, 4015 iterations)
Iteration:  4015   Scaled dual infeas =          2432.252301
Iteration:  4119   Scaled dual infeas =             0.247023
Iteration:  4120   Scaled dual infeas =             0.243853
Iteration:  4210   Scaled dual infeas =             0.000141
Iteration:  4236   Dual objective     = -55737664023976141127680.000000
Iteration:  4353   Dual objective     = -2867330273735413858304.000000
Iteration:  4415   Dual objective     = -2655849503210745626624.000000
Iteration:  4478   Dual objective     = -2606491316223142789120.000000
Iteration:  4521   Dual objective     = -2509897216374419750912.000000
Iteration:  4532   Dual objective     = -2503654119635770933248.000000
Iteration:  4632   Dual objective     = -2421218117263895822336.000000
Elapsed time = 23.84 sec. (60816.25 ticks, 4651 iterations)
Iteration:  4742

Iteration:  9154   Dual objective     = 1223383807091885473792.000000
Elapsed time = 185.81 sec. (256312.64 ticks, 9225 iterations)
Repairing basis singularity.
Added to 1 columns superbasic list.
Markowitz threshold set to 0.7
Iteration:  9254   Dual objective     = 1261947108043409326080.000000
Iteration:  9345   Dual objective     = 1292195978888383823872.000000
Iteration:  9445   Dual objective     = 1351839691647160418304.000000
Elapsed time = 195.20 sec. (266314.51 ticks, 9466 iterations)
Iteration:  9467   Dual objective     = 1385460190437518868480.000000
Iteration:  9493   Dual objective     = 1400561409600630030336.000000
Iteration:  9532   Dual objective     = 1428718601479000686592.000000
Iteration:  9651   Dual objective     = 1511545895548402532352.000000
Iteration:  9722   Dual objective     = 1649420320122680115200.000000
Elapsed time = 204.08 sec. (276326.73 ticks, 9775 iterations)
Iteration:  9785   Dual objective     = 1687211951952646111232.000000
Iteration:  9885  

Elapsed time = 438.94 sec. (523763.89 ticks, 16161 iterations)
Iteration: 16259   Dual objective     = 4377466190338503737344.000000
Iteration: 16280   Dual objective     = 4388779949609070886912.000000
Iteration: 16379   Dual objective     = 4434543131294722162688.000000
Iteration: 16391   Dual objective     = 4440706140438786473984.000000
Elapsed time = 447.75 sec. (534464.42 ticks, 16481 iterations)
Iteration: 16481   Dual objective     = 4501487926096860020736.000000
Iteration: 16522   Dual objective     = 4517250041760479969280.000000
Iteration: 16587   Dual objective     = 4572228770959859908608.000000
Iteration: 16687   Dual objective     = 4617126389996080594944.000000
Elapsed time = 457.14 sec. (544718.71 ticks, 16721 iterations)
Iteration: 16721   Dual objective     = 4663273240937294200832.000000
Iteration: 16736   Dual objective     = 4665900934268637937664.000000
Iteration: 16737   Dual objective     = 4672934886591606292480.000000
Iteration: 16837   Dual objective     = 4

### Save and summarize the results 💾📈

The "**save_results()**" function will save the results in an processed format to an excel file in the data folder that has been specified at the beginning.

In [ ]:
pulpo_worker.save_results(choices=choices, demand=demand, name='electricity_showcase_results.xlsx')

You can inspect the generated excel file.

There is another function to summarize the results in dataframe form within jupyter notbeooks called "summarize_results". This function has similar inputs to the "save_results" function, but does not require the specification of a filename. Additionally, by specifying the "*zeroes*" parameter to "*True*" all the not-selected choices are omitted in the summary.

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

# Closing Remarks

This is the end of the very basic PULPO functionalities using the electricity case study. 

The following sections will dive deeper into additional functionalities.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Additional Constraints</h2>
</div>

<div style="text-align: center;">
    <img src="pictures/electricity_showcase_2.png" alt="Image Alt Text" style="width: 50%;" />
</div>

#### Technosphere Flows

Let's assess what happens if the "_electricity production, nuclear, pressure water reactor | electricity, high voltage | DE_" activity is indirectly constrained trough a restriction on "_nuclear fuel element, for pressure water reactor, UO2 4.0% & MOX_"

In [ ]:
activities = ["market for nuclear fuel element, for pressure water reactor, UO2 4.0% & MOX"]
reference_products = ["nuclear fuel element, for pressure water reactor, UO2 4.0% & MOX"]
locations = ["GLO"]

nuclear_fuel = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

In [ ]:
nuclear_fuel

In [ ]:
upper_limit = {nuclear_fuel[0]: 100000}

The rationale behind choosing this activity and this limit is based on inspection of the scaling vector of the previous results. This activity is limiting for the nuclear electricity activity but not for the others, so, to enforce a different result than before, this activity is constrained.

In [ ]:
pulpo_worker.instantiate(choices=choices, demand=demand, upper_limit=upper_limit)
results = pulpo_worker.solve()

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, constraints=upper_limit)

As can be seen from the summary above, part of the final electricity demand is supplied by the wind turbine processes, because the nuclear process is constrained by the nuclear fuel process. It is also evident that the impact is higher than the previous one, as the GWP minimizing process (nuclear) can no longer supply the full demand.

#### Elementary Flows

Apart from technosphere flows, elementary flows may also be constrained. This could be a limitation on the extraction of raw materials or the emission of a certain compound. In this case study, we limit the emission of Radon-22:

In [25]:
elem = pulpo_worker.retrieve_envflows(activities='Radon-222')
elem

['Radon-222' (kilo Becquerel, None, ('air',)),
 'Radon-222' (kilo Becquerel, None, ('air', 'urban air close to ground')),
 'Radon-222' (kilo Becquerel, None, ('air', 'low population density, long-term')),
 'Radon-222' (kilo Becquerel, None, ('air', 'non-urban air or from high stacks')),
 'Radon-222' (kilo Becquerel, None, ('air', 'lower stratosphere + upper troposphere'))]

As can be seen from the results of the previous assessment, the "low population density, long-term" emission is the largest elementary flow with 3.69x1E13 kg Becquerel. For testing purpose, we limit this emissio to 1.00x1E13 kg Becquerel:

In [26]:
elem = [flow for flow in elem if 'long-term' in str(flow)]
elem_limit = {elem[0]: 1e13}
instance = pulpo_worker.instantiate(choices=choices, demand=demand, upper_elem_limit=elem_limit)
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH) 

Creating Instance
Instance created
GAMS solvers library availability: True
Solver path: C:\GAMS\37\gams.exe
        8.98 seconds required for presolve
--- Job model.gms Start 12/10/23 10:57:57 37.1.0 r07954d5 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\37\gmsprmNT.txt
    C:\Users\Usuario\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIP CPLEX
    Input C:\Users\Usuario\AppData\Local\Temp\tmpv8yghurp\model.gms
    Output C:\Users\Usuario\AppData\Local\Temp\tmpv8yghurp\output.lst
    ScrDir C:\Users\Usuario\AppData\Local\Temp\tmpv8yghurp\225a\
    SysDir C:\GAMS\37\
    CurDir C:\Users\Usuario\AppData\Local\Temp\tmpv8yghurp\
    LogOption 3
    OptCR 1E-9
Licensee: Antonio Espuna, Single User License            S210319|0002AN-WIN
          Universitat Politecnica de Catalunya, Chemical Engineering DC6757
          C:\Users\Usuario\Documents\GAMS\gamslice.txt
          antonio.espuna@upc.edu                                           
Processor information: 1 s

In [27]:
pulpo_worker.summarize_results(choices=choices, demand=demand)

The following demand / functional unit has been specified: 


,Demand
"market for electricity, high voltage | electricity, high voltage | DE",1.288190e+11



These are the impacts contained in the objective:


,Key,Value
0,"('IPCC 2013', 'climate change', 'GWP 100a')",1.716292e+10



The following choices were made: 
electricity


electricity  \
                                                                                            Activity   
Activity 0                          electricity production, lignite | electricity, high voltage | DE   
Activity 1                        electricity production, hard coal | electricity, high voltage | DE   
Activity 2  electricity production, nuclear, pressure water reactor | electricity, high voltage | DE   
Activity 3     electricity production, wind, 1-3MW turbine, onshore | electricity, high voltage | DE   

                                        
                Capacity         Value  
Activity 0  1.000000e+16  0.000000e+00  
Activity 1  1.000000e+16  0.000000e+00  
Activity 2  1.000000e+16  1.674902e+10  
Activity 3  1.000000e+16  6.677409e+10

No additional constraints have been passed.


As can be seen, the share of nuclear power is limited through the introduced elementary flow constraint.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Additional Methods</h2>
</div>

Let's see how to evaluate different methods and set them as objectives, in this case evaluating the ReCiPe endpoints, and setting the human health one as objective:

In [28]:
methods = {"('IPCC 2013', 'climate change', 'GWP 100a')": 0,
           "('ReCiPe Endpoint (E,A)', 'resources', 'total')": 0,
           "('ReCiPe Endpoint (E,A)', 'human health', 'total')": 1,
           "('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'total')": 0}

With this, a new Pulpo worker must be created:

In [29]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, methods, directory)

In [30]:
pulpo_worker.get_lci_data()

In [31]:
pulpo_worker.instantiate(choices=choices, demand=demand)
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

Creating Instance
Instance created
GAMS solvers library availability: True
Solver path: C:\GAMS\37\gams.exe
        8.91 seconds required for presolve
--- Job model.gms Start 12/10/23 10:58:28 37.1.0 r07954d5 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\37\gmsprmNT.txt
    C:\Users\Usuario\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIP CPLEX
    Input C:\Users\Usuario\AppData\Local\Temp\tmpuanwir_k\model.gms
    Output C:\Users\Usuario\AppData\Local\Temp\tmpuanwir_k\output.lst
    ScrDir C:\Users\Usuario\AppData\Local\Temp\tmpuanwir_k\225a\
    SysDir C:\GAMS\37\
    CurDir C:\Users\Usuario\AppData\Local\Temp\tmpuanwir_k\
    LogOption 3
    OptCR 1E-9
Licensee: Antonio Espuna, Single User License            S210319|0002AN-WIN
          Universitat Politecnica de Catalunya, Chemical Engineering DC6757
          C:\Users\Usuario\Documents\GAMS\gamslice.txt
          antonio.espuna@upc.edu                                           
Processor information: 1 s

In [32]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

The following demand / functional unit has been specified: 


,Demand
"market for electricity, high voltage | electricity, high voltage | DE",1.288190e+11



These are the impacts contained in the objective:


,Key,Value
0,"('ReCiPe Endpoint (E,A)', 'human health', 'total')",2.183741e+09



The following impacts were calculated: 


,Key,Value
0,"('IPCC 2013', 'climate change', 'GWP 100a')",1.739235e+10
1,"('ReCiPe Endpoint (E,A)', 'human health', 'total')",2.183741e+09
3,"('ReCiPe Endpoint (E,A)', 'resources', 'total')",9.161819e+08
2,"('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'total')",7.363601e+08



The following choices were made: 
electricity


electricity  \
                                                                                         Activity   
Activity 0  electricity production, wind, 1-3MW turbine, onshore | electricity, high voltage | DE   

                                        
                Capacity         Value  
Activity 0  1.000000e+16  8.350255e+10

No additional constraints have been passed.


From the summary above, it can be seen that for the "_human health_" category, the nuclear process is not the most suitable anymore. With this objective, the wind turbine process is selected. 

As another category is minimized, the GWP has changed as well: previously, with nuclear electricity the total GWP was 1.599836e+10 while with wind electricity it is 1.739234e+10.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Lower Level Decisions</h2>
</div>

<div style="text-align: center;">
    <img src="pictures/electricity_showcase_3.png" alt="Image Alt Text" style="width: 50%;" />
</div>

In this case study, we would like to keep the current share of the electricity supplied by fossil sources the same. The choices that we consider on the electricity production level are between the coal and lignite activities:

In [33]:
methods = {"('IPCC 2013', 'climate change', 'GWP 100a')": 1,
           "('ReCiPe Endpoint (E,A)', 'resources', 'total')": 0,
           "('ReCiPe Endpoint (E,A)', 'human health', 'total')": 0,
           "('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'total')": 0}

In [34]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, methods, directory)
pulpo_worker.get_lci_data()

In [35]:
activities = ["electricity production, lignite", 
             "electricity production, hard coal"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]

electricity_activities = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

Instead of assessing only the **technology** choices, we are invetigating the best **regional** choice for the source of coal and lignite:

In [36]:
coal_activities = ["market for hard coal"]
lignite_activities = ["market for lignite"]

coal_activities = pulpo_worker.retrieve_activities(activities=coal_activities)
lignite_activities = pulpo_worker.retrieve_activities(activities=lignite_activities)

The updated choice dictionary looks like this:

In [37]:
choices  = {'electricity': {elec: 1e16 for elec in electricity_activities},
            'coal': {coal: 1e16 for coal in coal_activities},
            'lignite': {lignite: 1e16 for lignite in lignite_activities}}

Instantiating and solving the adapted problem:

In [38]:
pulpo_worker.instantiate(choices=choices, demand=demand)
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

Creating Instance
Instance created
GAMS solvers library availability: True
Solver path: C:\GAMS\37\gams.exe
        9.14 seconds required for presolve
--- Job model.gms Start 12/10/23 10:59:06 37.1.0 r07954d5 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\37\gmsprmNT.txt
    C:\Users\Usuario\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIP CPLEX
    Input C:\Users\Usuario\AppData\Local\Temp\tmphbzwi0xr\model.gms
    Output C:\Users\Usuario\AppData\Local\Temp\tmphbzwi0xr\output.lst
    ScrDir C:\Users\Usuario\AppData\Local\Temp\tmphbzwi0xr\225a\
    SysDir C:\GAMS\37\
    CurDir C:\Users\Usuario\AppData\Local\Temp\tmphbzwi0xr\
    LogOption 3
    OptCR 1E-9
Licensee: Antonio Espuna, Single User License            S210319|0002AN-WIN
          Universitat Politecnica de Catalunya, Chemical Engineering DC6757
          C:\Users\Usuario\Documents\GAMS\gamslice.txt
          antonio.espuna@upc.edu                                           
Processor information: 1 s

Visualizing the results

In [39]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

The following demand / functional unit has been specified: 


,Demand
"market for electricity, high voltage | electricity, high voltage | DE",1.288190e+11



These are the impacts contained in the objective:


,Key,Value
0,"('IPCC 2013', 'climate change', 'GWP 100a')",6.160225e+10



The following impacts were calculated: 


...

coal


coal                            
                                         Activity      Capacity         Value
Activity 0  market for hard coal | hard coal | ZA  1.000000e+16  2.189073e+10

lignite


lignite                            
                                      Activity      Capacity         Value
Activity 0  market for lignite | lignite | RER  1.000000e+16  1.884971e+09

No additional constraints have been passed.


I can be seen that out of the fossil alternatives, the electricity from coal minimizes GWP when the coal comes from RLA [Latin America and the Caribbean] (omitting transport emissions). Moreover, it can be seen that the market for lignite is somewhere used upstream of the coal production activity. In the place where it is used, the RER market for lignite is the preferred one. The lignite consumption is one order of magnitude lower than the coal consumption.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Supply vs. Demand Problem</h2>
</div>

Finally, let's test and assess the functionality of PULPO to specify supply values rather than demand values. This can be done by setting the lower_limit and the upper_limit of activities to the same value. This will enforce the corresponding scaling vector entry of that activity to the specified value, and activates the slack variable to relax the demand value. 

This can simply be done by specifying the upper and lower limits rather than the demand (note, we continue with the choices from the previous section):

In [40]:
upper_limit = {electricity_market[0]: 1.28819e+11}
lower_limit = {electricity_market[0]: 1.28819e+11}

In [41]:
pulpo_worker.instantiate(choices=choices, upper_limit=upper_limit, lower_limit=lower_limit)
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

Creating Instance
Instance created
GAMS solvers library availability: True
Solver path: C:\GAMS\37\gams.exe
        8.92 seconds required for presolve
--- Job model.gms Start 12/10/23 10:59:35 37.1.0 r07954d5 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\37\gmsprmNT.txt
    C:\Users\Usuario\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIP CPLEX
    Input C:\Users\Usuario\AppData\Local\Temp\tmp8x7vdfur\model.gms
    Output C:\Users\Usuario\AppData\Local\Temp\tmp8x7vdfur\output.lst
    ScrDir C:\Users\Usuario\AppData\Local\Temp\tmp8x7vdfur\225a\
    SysDir C:\GAMS\37\
    CurDir C:\Users\Usuario\AppData\Local\Temp\tmp8x7vdfur\
    LogOption 3
    OptCR 1E-9
Licensee: Antonio Espuna, Single User License            S210319|0002AN-WIN
          Universitat Politecnica de Catalunya, Chemical Engineering DC6757
          C:\Users\Usuario\Documents\GAMS\gamslice.txt
          antonio.espuna@upc.edu                                           
Processor information: 1 s

In [42]:
pulpo_worker.summarize_results(choices=choices, demand=demand, constraints=upper_limit, zeroes=True)

The following demand / functional unit has been specified: 


,Demand
"market for electricity, high voltage | electricity, high voltage | DE",1.288190e+11



These are the impacts contained in the objective:


,Key,Value
0,"('IPCC 2013', 'climate change', 'GWP 100a')",5.890743e+10



The following impacts were calculated: 


...

coal                            
                                         Activity      Capacity         Value
Activity 0  market for hard coal | hard coal | ZA  1.000000e+16  2.093311e+10

lignite


lignite                            
                                      Activity      Capacity         Value
Activity 0  market for lignite | lignite | RER  1.000000e+16  1.802512e+09


The following constraints were implemented and oblieged: 


,Constraints
"market for electricity, high voltage | electricity, high voltage | DE",1.288190e+11


From the results it can be observed that the resulting GWP is **considerably** lower (6.191744e+10 vs. 5.920967e+10: ~4.4%) than in the previous section. Now, the production value (supply) of electricity is specified, so that electricity consumed in the background is accounted for in the specifications.

Overall, when specifying supply values instead of demand values, the corresponding scaling vector entries are always smaller.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Foreground vs. Background Modelling</h2>
</div>

<div style="text-align: center;">
    <img src="pictures/electricity_showcase_4.png" alt="Image Alt Text" style="width: 50%;" />
</div>

In this final part of the electricity showcase, the difference between foreground and background modelling and optimization is demonstrated. For that purpose a foreground system must be created ... this can be done either by hand or I can upload a folded foreground system for this case ... in essence, a new database must be created (e.g. "_cutoff38_foreground_" where copies from the market and electricity production activites are duplicated to from the "_cutoff38_". What this does is disconnecting this activities from their downstream.

We are then replacing the original inputs in the electricity market with the duplicated processes from the foreground system, effectively disconnecting the choices made in the electricity market from the upstream of the electricity production technologies:

With this in mind, we can set up the optimization as usual, importing the "_cutoff38_foreground_" database instead of the "_cutoff38_" database:

In [43]:
project = "pulpo"
database = "cutoff38_foreground"

In [44]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, methods, directory)

In [45]:
pulpo_worker.get_lci_data()

Now, choose the foreground market for electricity:

In [46]:
activities = ["market for electricity, high voltage, foreground"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]

In [47]:
foreground_market = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

In [48]:
demand = {foreground_market[0]: 1.28819e+11}

And the foreground electricity production technologies:

In [49]:
activities = ["electricity production, lignite, foreground", 
             "electricity production, hard coal, foreground",
             "electricity production, nuclear, pressure water reactor, foreground",
             "electricity production, wind, 1-3MW turbine, onshore, foreground"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]

electricity_activities = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

In [50]:
choices  = {'electricity': {electricity_activities[0]: 1e16,
                         electricity_activities[1]: 1e16,
                         electricity_activities[2]: 1e16,
                         electricity_activities[3]: 1e16}}

Create the instance and solve the problem:

In [51]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand)

Creating Instance
Instance created


In [52]:
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

GAMS solvers library availability: True
Solver path: C:\GAMS\37\gams.exe
        9.41 seconds required for presolve
--- Job model.gms Start 12/10/23 11:00:07 37.1.0 r07954d5 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\37\gmsprmNT.txt
    C:\Users\Usuario\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIP CPLEX
    Input C:\Users\Usuario\AppData\Local\Temp\tmpfm76rsg8\model.gms
    Output C:\Users\Usuario\AppData\Local\Temp\tmpfm76rsg8\output.lst
    ScrDir C:\Users\Usuario\AppData\Local\Temp\tmpfm76rsg8\225a\
    SysDir C:\GAMS\37\
    CurDir C:\Users\Usuario\AppData\Local\Temp\tmpfm76rsg8\
    LogOption 3
    OptCR 1E-9
Licensee: Antonio Espuna, Single User License            S210319|0002AN-WIN
          Universitat Politecnica de Catalunya, Chemical Engineering DC6757
          C:\Users\Usuario\Documents\GAMS\gamslice.txt
          antonio.espuna@upc.edu                                           
Processor information: 1 socket(s), 12 core(s), and 20 thread

In [53]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

The following demand / functional unit has been specified: 


,Demand
"market for electricity, high voltage, foreground | electricity, high voltage | DE",1.288190e+11



These are the impacts contained in the objective:


,Key,Value
0,"('IPCC 2013', 'climate change', 'GWP 100a')",1.764966e+10



The following impacts were calculated: 


,Key,Value
0,"('IPCC 2013', 'climate change', 'GWP 100a')",1.764966e+10
1,"('ReCiPe Endpoint (E,A)', 'human health', 'total')",2.927733e+09
3,"('ReCiPe Endpoint (E,A)', 'resources', 'total')",7.551099e+08
2,"('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'total')",7.155927e+08



The following choices were made: 
electricity


electricity  \
                                                                                                        Activity   
Activity 0  electricity production, nuclear, pressure water reactor, foreground | electricity, high voltage | DE   

                                        
                Capacity         Value  
Activity 0  1.000000e+16  8.153354e+10

No additional constraints have been passed.


The result of this optimization is a minimum GWP of 1.764977e+10 kg CO2eq instead of 1.599836e+10 kg CO2eq from the initial calculations uising the full LCI. This is a difference of **8%**!

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Integer Cuts</h2>
</div>

This functionality will be available in future versions of PULPO